# 선형 독립

벡터집합 $V$가 $\{V_1, V_2, V_3, ... V_n\}$ 일 때, $C_1 = C_2 = C_3 = .... = C_N = 0$으로 유일한 해를 가지면 벡터 집합 $V$는 선형독립이라고 합니다.

$$
\begin{matrix}
c_1V_1 + c_2V_2 + c_3V_3 + .... + c_nV_n = 0
\end{matrix}
$$

이것을 아주 간단하게 주장하자면, 선형독립은 벡터집합 $V$의 어떠한 원소도 나머지 원소들의 선형결합으로 나타낼 수 없습니다. 즉, 각각의 벡터는 서로 독립적으로 존재한다고 할 수 있습니다. 반면, 선형종속의 의미​는 벡터집합 S의 원소 중 적어도 하나는 나머지 원소들의 선형결합으로 나타낼 수 있습니다. 즉, 어떠한 특정 벡터가 나머지 벡터에 의존하여 영향을 받는다고 할 수 있습니다.

## 예제 1:  Vectors in $\mathbb{R}^2$

$$
\begin{matrix}
V_1 = \left[ \begin{array}{r} 2 \\ 1 \end{array}\right] \hspace{1cm} 
V_2 = \left[ \begin{array}{r} 1 \\ -6  \end{array}\right] \hspace{1cm}
\end{matrix}
$$

이 벡터 집합이 선형적으로 독립적인지 확인하려면 다음 벡터 방정식을 살펴봐야 합니다.

$$
\begin{matrix}
c_1\left[ \begin{array}{r} 2 \\ 1 \end{array}\right] +
c_2\left[ \begin{array}{r} 1 \\ -6  \end{array}\right] =
\left[ \begin{array}{r} 0 \\ 0 \end{array}\right]
\end{matrix}
$$



In [1]:
import numpy as np

def row_swap(A,k,l):
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A).astype('float64')

    for j in range(n):
        temp = B[k][j]
        B[k][j] = B[l][j]
        B[l][j] = temp
        
    return B

def row_scale(A,k,scale):
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A).astype('float64')

    for j in range(n):
        B[k][j] *= scale
        
    return B

def row_add(A,k,l,scale):
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A).astype('float64')
        
    for j in range(n):
        B[l][j] += B[k][j]*scale
        
    return B

def full_row_reduction(A, tol = 1e-14):
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A

    B = np.copy(A).astype('float64')

    # Set initial pivot search position
    pivot_row = 0
    pivot_col = 0
    
    # Continue steps of elimination while possible pivot positions are 
    # within bounds of the array.
    
    while(pivot_row < m and pivot_col < n):

        # Set pivot value to current pivot position
        pivot = B[pivot_row,pivot_col]
        
        # If pivot is zero, search down current column, and then subsequent
        # columns (at or beyond pivot_row) for the next nonzero entry in the 
        # array is found, or the last entry is reached.

        row_search = pivot_row
        col_search = pivot_col
        search_end = False

        while(pivot == 0 and not search_end):
            if(row_search < m-1):
                row_search += 1
                pivot = B[row_search,col_search]
            else:
                if(col_search < n-1):
                    row_search = pivot_row
                    col_search += 1
                    pivot = B[row_search,col_search]
                else:  
                    # col_search = n-1 and row_search = m-1
                    search_end = True
                        
        # Swap row if needed to bring pivot to position for rref
        if (pivot != 0 and pivot_row != row_search):
            B = row_swap(B,pivot_row,row_search)
            pivot_row, row_search = row_search, pivot_row
            
        # Set pivot position to search position
        pivot_row = row_search
        pivot_col = col_search
            
        # If pivot is nonzero, carry on with elimination in pivot column 
        if (pivot != 0):
            
            # Set pivot entry to one
            B = row_scale(B,pivot_row,1./B[pivot_row,pivot_col])

            # Create zeros above pivot
            for i in range(pivot_row):    
                B = row_add(B,pivot_row,i,-B[i][pivot_col])
                # Force known zeros
                B[i,pivot_col] = 0

            # Create zeros below pivot
            for i in range(pivot_row+1,m):    
                B = row_add(B,pivot_row,i,-B[i][pivot_col])
                # Force known zeros
                B[i,pivot_col] = 0

            # Force small numbers to zero to account for roundoff error
            for i in range(m):
                for j in range(n):
                    if abs(B[i,j])< tol :
                        B[i,j] = 0

        # Advance to next possible pivot position
        pivot_row += 1
        pivot_col += 1
        
    return B

def row_reduction(A):
    
    m = A.shape[0]  # A has m rows 
    n = A.shape[1]  # It is assumed that A has m+1 columns
    
    B = np.copy(A).astype('float64')

    # For each step of elimination, we find a suitable pivot, move it into
    # position and create zeros for all entries below.
    
    for k in range(m):
        # Set pivot as (k,k) entry
        pivot = B[k][k]
        pivot_row = k
        
        # Find a suitable pivot if the (k,k) entry is zero
        while(pivot == 0 and pivot_row < m-1):
            pivot_row += 1
            pivot = B[pivot_row][k]
            
        # Swap row if needed
        if (pivot_row != k):
            B = row_swap(B,k,pivot_row)
            
        # If pivot is nonzero, carry on with elimination in column k
        if (pivot != 0):
            B = row_scale(B,k,1./B[k][k])
            for i in range(k+1,m):    
                B = row_add(B,k,i,-B[i][k])
        else:
            print("Pivot could not be found in column",k,".")
            
    return B



In [2]:
import numpy as np

A_augmented = np.array([[2,1,0],[1,-6,0]])
print(full_row_reduction(A_augmented))

[[1. 0. 0.]
 [0. 1. 0.]]


RREF에서 방정식의 유일한 해는 $c_1 = c_2 = 0$임을 알 수 있습니다. 집합 $\{V_1, V_2\}$는 선형 독립적입니다.

## 예제 2: Vectors in $\mathbb{R}^3$

$$
\begin{matrix}
W_1 = \left[ \begin{array}{r} 2 \\ -1  \\ 1 \end{array}\right] \hspace{1cm} 
W_2 = \left[ \begin{array}{r} 1 \\ -4 \\ 0  \end{array}\right] \hspace{1cm}
W_3 = \left[ \begin{array}{r} 3 \\ 2 \\ 2  \end{array}\right] \hspace{1cm}
\end{matrix}
$$

다시 한 번 벡터 방정식의 해를 살펴봐야 합니다.

$$
\begin{matrix}
c_1\left[ \begin{array}{r} 2 \\ -1 \\ 1 \end{array}\right] +
c_2\left[ \begin{array}{r} 1 \\ -4 \\ 0  \end{array}\right] +
c_3\left[ \begin{array}{r} 3 \\ 2 \\ 2  \end{array}\right] =
\left[ \begin{array}{r} 0 \\ 0 \\ 0\end{array}\right]\end{matrix}
$$


In [3]:
B_augmented = np.array([[2,1,3,0],[-1,-4,2,0],[1,0,2,0]])
print(full_row_reduction(B_augmented))

[[ 1.  0.  2.  0.]
 [ 0.  1. -1.  0.]
 [ 0.  0.  0.  0.]]


이 경우 $c_3$이 자유 변수임을 알 수 있습니다. 만약 $c_3 = 1$로 설정하면 $c_2 = 1$, $c_1 = -2$가 됩니다.  $c_1 = c_2 = c_3 = 0$ 이외의 해를 찾을 수 있으므로 벡터 집합 $\{W_1, W_2, W_3\}$은 선형 종속적입니다.

## 동형 시스템

선형시스템은 행렬 방정식 $AX = 0$으로 설명할 수 있는 경우 **동형 시스템**이라고 합니다. 이러한 시스템의 해는 $AX=B$ 시스템의 해와 연결되어 있습니다. 동형 시스템은 선형 독립성의 개념과도 관련이 있습니다. 이 모든 아이디어를 서로 연결하면 선형 독립성에 대한 정보를 바탕으로 $AX=B$ 시스템의 해에 대한 정보를 얻을 수 있습니다.

이전 예제에서는 벡터 방정식 $c_1V_1 + c_2V_2 + c_3V_3 + .... + c_nV_n = 0$ 의 벡터 집합 $\{V_1, V_2, V_3 .... V_n\}$ 벡터 집합이 선형적으로 독립적인지 확인합니다. 이 벡터 방정식은 $AX=0$로도 설명할 수 있는  동차 선형 시스템을 나타내며, 여기서 $V_1$, $V_2$, ... $V_n$은 행렬 $A$의 열이고, $X$는 미지 계수의 벡터입니다. 벡터의 집합은 연관된 동질계가 모든 항목이 0인 벡터 이외의 해를 갖는 경우에만 선형 의존적입니다. 모든 0의 벡터를 **삼차 해법**이라고 합니다. 이 0 벡터는 $A$의 엔트리에 관계없이 *모든 동형계* $AX=0$에 대한 해이기 때문에 사칙연산 해라고 부릅니다. 이러한 이유로 우리는 $AX=0$에 대한 *비소수 해*의 존재 여부에만 관심이 있습니다.

동형 시스템 $AX=0$에 비동형 해가 있다고 가정하고, 이를 $X_h$라고 부를 수 있다고 가정해 봅시다. 또한 관련 비동차 시스템인 $AX=B$에도 어떤 특정 해가 있으며, 이를 $X_p$라고 부를 수 있다고 가정해 봅시다.  따라서 $AX_h = 0$과 $AX_p = B$가 됩니다.  이제 행렬 곱셈의 성질에 따라 $A(X_p + X_h) = AX_p + AX_h = B + 0$이므로 $X_p + X_h$도 $AX=B$의 해가 됩니다.

다음 시스템을 예로 들어 보겠습니다.

$$
\begin{matrix}
AX = \left[ \begin{array}{rrr} -3 & 2 & 1 \\ -2 & 1 & -1 \\ 4 & 3 & 3 \end{array}\right]
\left[ \begin{array}{r} x_1 \\ x_2 \\ x_3 \end{array}\right]=
\left[ \begin{array}{r} -6 \\ 1 \\ 13  \end{array}\right]= B
\end{matrix}
$$

연관된 동질계를 살펴보고 $A$의 열이 선형적으로 독립적인지 확인할 수 있습니다.

$$
\begin{matrix}
\left[ \begin{array}{rrr} -3 & 2 & 1 \\ -2 & 1 & -1 \\ 4 & 3 & 3 \end{array}\right]
\left[ \begin{array}{r} c_1 \\ c_2 \\ c_3 \end{array}\right]=
\left[ \begin{array}{r} 0 \\ 0 \\ 0  \end{array}\right]
\end{matrix}
$$

In [4]:
A_augmented = np.array([[-3,2,1,0],[-2,1,-1,0],[4,-3,-3,0]])
A_augmented_reduced = full_row_reduction(A_augmented)
print(A_augmented_reduced)

[[1. 0. 3. 0.]
 [0. 1. 5. 0.]
 [0. 0. 0. 0.]]


세 번째 열에 피벗이 없다는 것은 $c_3$이 자유 변수이며, 동종 시스템에 대한 비분산 해가 존재한다는 것을 나타냅니다.  한 가지 가능성은 $c_1 = 3$, $c_2=5$, $c_3 = -1$입니다.  여기서 모든 항목이 0이기 때문에 증강 행렬의 마지막 열에서 행 연산을 수행할 필요가 없다는 점에 주목할 필요가 있습니다.  동종 시스템을 고려할 때는 계수 행렬의 RREF를 구하는 것으로 충분합니다.

동차방정식이 비미분 해를 갖는다는 사실은 $A$의 열을 벡터로 생각하면 선형적으로 의존한다는 것을 의미합니다.  지금까지의 논의를 바탕으로 $AX=B$ 시스템에 해가 있다면, 그 해는 고유하지 않을 것으로 예상할 수 있습니다.

In [5]:
A_augmented = np.array([[-3,2,1,-6],[-2,1,-1,1],[4,-3,-3,13]])
A_augmented_reduced = full_row_reduction(A_augmented)
print(A_augmented_reduced)

[[  1.   0.   3.  -8.]
 [  0.   1.   5. -15.]
 [  0.   0.   0.   0.]]


RREF는 두 개의 방정식을 나타내며, 세 번째 변수는 자유 변수입니다.

$$
\begin{matrix}
x_1 \quad\quad + 3x_3 & = & -8\\
x_2 + 5x_3 & = & = -15 
\end{matrix}
$$

가능한 해를 표현하기 위해 $x_3 = t$를 설정하면 $x_2 = -15 -5t$와 $x_1 = -8-3t$가 됩니다.  이러한 구성 요소는 매개변수 $t$를 포함하는 벡터로 조립할 수 있습니다.

$$
\begin{matrix}
X = \left[ \begin{array}{c} -8-3t \\ -15-5t  \\ t \end{array}\right]
\end{matrix}
$$

이 벡터를 두 조각으로 나누면 이 솔루션을 동종 시스템의 솔루션과 연결하는 데 도움이 됩니다.

$$
\begin{matrix}
X  = 
\left[ \begin{array}{c} -8 \\ -15  \\ 0 \end{array}\right] + 
t\left[ \begin{array}{c} -3 \\ -5  \\ 1 \end{array}\right] = X_p + X_h
\end{matrix}
$$

모든 $t$에 대해 $AX_p= B$, $AX_h= 0$, 모든 $t$에 대해 $A(X_p+X_h) = B$라는 것을 확인할 수 있습니다.

In [6]:
A = np.array([[-3,2,1],[-2,1,-1],[4,-3,-3]])

X_p = np.array([[-8],[-15],[0]])
X_h = np.array([[-3],[-5],[1]])

t = np.random.rand()
X = X_p + t*X_h

print(X)
print('\n')
print(A@X_p)
print('\n')
print(A@(t*X_h))
print('\n')
print(A@X)

[[ -8.44841871]
 [-15.74736452]
 [  0.1494729 ]]


[[-6]
 [ 1]
 [13]]


[[0.00000000e+00]
 [0.00000000e+00]
 [1.11022302e-16]]


[[-6.]
 [ 1.]
 [13.]]


동종 시스템의 증강 행렬에 대한 RREF의 처음 세 열이 $AX=B$ 시스템의 RREF와 정확히 동일하다는 점에 유의해야 합니다. 물론 처음 세 열은 두 시스템에서 동일한 계수 행렬 $A$에서 나오기 때문에 그럴 수밖에 없습니다. 여기서 중요한 점은 $AX=B$ 시스템은 $A$의 열이 선형적으로 독립적인 경우에만 고유한 해를 가질 수 있다는 것입니다.

## Null space

동종 시스템의 개념이 정립되었으므로 이제 두 번째 기본 부분 공간을 정의할 준비가 되었습니다. A$가 $m\times n$ 행렬인 경우, $A$의 **널 공간**은 $AX=0$이 되는 벡터 $X$의 집합입니다. 다시 말해, $A$의 널 공간은 동종 시스템 $AX=0$에 대한 모든 해의 집합입니다. $A$의 널 공간은 $\mathbb{R}^n$의 부분공간이며, $\mathcal{N}(A)$라는 표기법으로 쓰입니다. 이제 앞서 설명한 내용을 널 공간의 관점에서 재구성할 수 있습니다.  

- 행렬 $A$의 열은 $\mathcal{N}(A)$가 0 벡터만 포함하는 경우에만 선형적으로 독립적입니다.

- $AX=B$ 시스템은 $\mathcal{N}(A)$ 가 0 벡터만 포함하는 경우에만 최대 하나의 해를 갖습니다.

$A$의 기본 부분공간과 $AX=B$ 시스템의 해집합을 연결하면 선형 시스템에 대한 이해와 이를 풀 수 있는 방법을 더욱 강화하는 일반적인 결론을 내릴 수 있습니다.